In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [3]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

search_query = "atelectasis"
URL = f"https://radiopaedia.org/search?lang=gb&q={search_query}&scope=articles"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
radio_address = "https://radiopaedia.org"
all_address = [
    f"{radio_address}{e['href']}"
    for e in soup.find_all("a", class_="search-result search-result-article")
]
all_address

['https://radiopaedia.org/articles/atelectasis-summary?lang=gb',
 'https://radiopaedia.org/articles/linear-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/lung-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/subsegmental-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/round-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/passive-atelectasis-1?lang=gb',
 'https://radiopaedia.org/articles/compressive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/cicatrisation-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/segmental-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/adhesive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/resorptive-obstructive-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/chronic-maxillary-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/middle-ear-atelectasis?lang=gb',
 'https://radiopaedia.org/articles/gravity-dependent-atelectasis-1?lang=gb',
 'https://radiopaedia.org/articles/c

In [4]:
from langchain.document_loaders import DiffbotLoader

raw_documents = DiffbotLoader(
    urls=[all_address[0]],
    api_token=DIFFBOT_API_TOKEN,
).load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [5]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()

In [6]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [7]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

In [8]:
chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis is a condition that describes small areas of collapsed lung. It can be caused by adjacent compression, passive atelectasis, or dependent atelectasis. The presentation of atelectasis can vary depending on the underlying cause, but it is usually characterized by breathlessness. Imaging, such as chest x-rays or CT scans, can confirm the presence of atelectasis and help determine the cause. Treatment involves managing the underlying cause and promoting deeper breathing if hypoventilation is a factor. \n',
 'sources': 'https://radiopaedia.org/articles/atelectasis-summary?lang=gb'}

In [9]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis can vary depending on the underlying cause. Common symptoms include breathlessness, but it can also be asymptomatic. The presentation of symptoms also depends on the underlying cause. \n',
 'sources': 'https://radiopaedia.org/articles/atelectasis-summary?lang=gb'}

In [10]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by adjacent compression (e.g. lung tumor, dilated aorta, large osteophyte), passive atelectasis (e.g. pleural effusion, pneumothorax), and dependent atelectasis (in the posterior portions of the lung due to patients not fully expanding their lungs while lying for long periods). \n',
 'sources': 'https://radiopaedia.org/articles/atelectasis-summary?lang=gb'}

In [11]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'For diagnosing atelectasis, radiologists need to look for small volume linear shadows on chest x-rays, usually peripherally or at the lung bases. The underlying cause may also be visible on the x-ray. CT scans can provide a more accurate depiction of the involved lung and can be helpful in determining the cause or confirming the absence of proximal obstruction. However, most cases of atelectasis are diagnosed and managed without CT investigation.\n',
 'sources': 'https://radiopaedia.org/articles/atelectasis-summary?lang=gb'}

In [12]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiographic features of atelectasis on a chest x-ray include small volume linear shadows, usually peripherally or at the lung bases. Lobar collapse will have a more typical appearance based on the lobe involved, whereas atelectasis can be more eccentric in position and appearance. CT chest allows for a more accurate depiction of the involved lung and can be helpful in determining the cause or confirming the absence of proximal obstruction. However, most cases of atelectasis are diagnosed and managed without CT investigation.\n',
 'sources': 'https://radiopaedia.org/articles/atelectasis-summary?lang=gb'}